Here I wanted to test and compare error rates for 2 approaches to calculate unique queries for different number of total/unique queries and hash size.

In [26]:
import numpy as np

In [111]:
total_queries = 500000
unique_queries = 100000
hash_size = 1000000
queries = np.random.choice([str(i) for i in range(unique_queries)], total_queries)

First approach is use a single byte to store if a remainder is present. It is not very memory efficient, but a bit easier to implement. Even with this approach, if hash table size is 1,000,000 we get error rate just a little bit more than the threshold of 5%.

In [112]:
hashes = bytearray(hash_size)
for i in range(total_queries):
    hashes[hash(queries[i]) % hash_size] = 1

print(f'error {(unique_queries - hashes.count(1)) / unique_queries * 100:.2f}%')

error 5.42%


The second approach is to use each of 8 bits of a single array element. It is 8 times more memory efficient.

In [113]:
hashes = bytearray(hash_size)
for i in range(total_queries):
    h = hash(queries[i]) % (hash_size * 8)
    hashes[h // 8] |= (1 << h % 8)

cnt = 0
for h in hashes:
    while h > 0:
        if h & 1:
            cnt += 1
        h >>= 1
        
print(f'error {(unique_queries - cnt) / unique_queries * 100:.2f}%')

error 1.23%


Thus we get much lower error rate using the same amount of memory. Even 200,000 bytes will be enough to get error rate below 5%.